## 경로 설정
생성하는 파일들
1. frame 폴더   : MP4의 모든 frame을 저장 (2160p)
2. JSON 폴더    : sapiens 모델 실행 결과 저장되는 keypoints 위치 json 파일로 저장
3. Vis 폴더     : 원본 Frame image에 json 결과 overlay한 결과
4. OUTPUT 폴더  : 최종 MP4 파일 위치 

In [1]:
import os, sys, torch
import mmpose, mmdet, mmengine, mmcv
from pathlib import Path

print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("mmpose:", mmpose.__version__, "| mmdet:", mmdet.__version__, "| mmengine:", mmengine.__version__, "| mmcv:", mmcv.__version__)

# ← 여기에 사용할 mp4 이름만 바꾸세요 (확장자/경로 제외, 파일명만)
mp4_name = "M01_VISIT2_상지"    # 예시

paths_tpl = {
    "MP4_PATH":    "../../../../data/김원 보산진 연구/{mp4_name}.MP4",
    "FRAME_DIR":   "../data/Patient_data/new_code/frames/{mp4_name}_frame",
    "JSON_DIR":    "../data/Patient_data/new_code/json/{mp4_name}_json",
    "VIS_DIR":     "../data/Patient_data/new_code/vis/{mp4_name}_vis",
    "OUTPUT_MP4":  "../data/Patient_data/new_code/output/{mp4_name}_output.mp4",
    "DET_CONFIG":  "../sapiens/pose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person_no_nms.py",
    "DET_CKPT":    "../sapiens/pose/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth",
    "POSE_CONFIG": "../sapiens/pose/configs/sapiens_pose/coco/sapiens_0.3b-210e_coco-1024x768.py",
    "POSE_CKPT":   "../sapiens/pose/checkpoints/sapiens_0.3b/sapiens_0.3b_coco_best_coco_AP_796.pth",
}

# {mp4_name} 치환
paths = {k: v.format(mp4_name=mp4_name) for k, v in paths_tpl.items()}

# 디렉토리는 생성, 파일은 존재 여부만 확인
dir_keys  = ["FRAME_DIR", "JSON_DIR", "VIS_DIR"]
file_keys = ["MP4_PATH", "OUTPUT_MP4", "DET_CONFIG", "DET_CKPT", "POSE_CONFIG", "POSE_CKPT"]

for k in dir_keys:
    Path(paths[k]).mkdir(parents=True, exist_ok=True)

print("\n[PATH CHECK]")
for k in dir_keys + file_keys:
    p = paths[k]
    exists = Path(p).exists()
    typ = "DIR " if k in dir_keys else "FILE"
    print(f"{k:12s} | {typ} | {'OK' if exists else 'MISSING'} | {p}")

# 사용 예시:
# cap = cv2.VideoCapture(paths["MP4_PATH"])
# output_mp4 = paths["OUTPUT_MP4"]
# frame_dir  = paths["FRAME_DIR"]
# json_dir   = paths["JSON_DIR"]
# vis_dir    = paths["VIS_DIR"]


CUDA available: True
GPU count: 1
mmpose: 1.3.2 | mmdet: 3.2.0 | mmengine: 0.10.7 | mmcv: 2.1.0

[PATH CHECK]
FRAME_DIR    | DIR  | OK | ../data/Patient_data/new_code/frames/M01_VISIT2_상지_frame
JSON_DIR     | DIR  | OK | ../data/Patient_data/new_code/json/M01_VISIT2_상지_json
VIS_DIR      | DIR  | OK | ../data/Patient_data/new_code/vis/M01_VISIT2_상지_vis
MP4_PATH     | FILE | OK | ../../../../data/김원 보산진 연구/M01_VISIT2_상지.MP4
OUTPUT_MP4   | FILE | MISSING | ../data/Patient_data/new_code/output/M01_VISIT2_상지_output.mp4
DET_CONFIG   | FILE | OK | ../sapiens/pose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person_no_nms.py
DET_CKPT     | FILE | OK | ../sapiens/pose/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
POSE_CONFIG  | FILE | OK | ../sapiens/pose/configs/sapiens_pose/coco/sapiens_0.3b-210e_coco-1024x768.py
POSE_CKPT    | FILE | OK | ../sapiens/pose/checkpoints/sapiens_0.3b/sapiens_0.3b_coco_best_coco_AP_796.pth


## Frame 추출
설정할 수 있는 것
1. 시간             : 초반 몇 초를 뽑을 것인가

In [30]:
import os
import cv2
from tqdm import tqdm

# 입력 비디오와 출력 폴더
DATA_DIR = r"../../../../data/김원 보산진 연구/M04_VISIT2.MP4"
OUTPUT_DIR = r"../data/test_patient_image/new_code/M04VISIT12_ori_frame"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 비디오 열기
cap = cv2.VideoCapture(DATA_DIR)
if not cap.isOpened():
    raise RuntimeError(f"비디오를 열 수 없습니다: {DATA_DIR}")

# FPS와 총 프레임 수 읽기
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT) > 0 else -1

# 저장할 프레임 수 계산 (앞 20초)
if fps and fps > 0:
    target_frames = int(round(30 * fps))
    if frame_count > 0:
        target_frames = min(target_frames, frame_count)
else:
    # FPS를 얻지 못한 경우: 시간 기준으로 루프에서 20초를 넘지 않게 처리
    target_frames = None

print(f"FPS: {fps:.3f} | 총 프레임: {frame_count} | 저장 예정 프레임: {target_frames if target_frames is not None else '시간 기준'}")

saved = 0
idx = 0

# 진행률 바 설정
pbar_total = target_frames if target_frames is not None else 100  # 대략 값
pbar = tqdm(total=pbar_total, desc="Extracting first 20s frames", unit="frame")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 현재 시간(ms) 확인 (FPS가 불확실할 때 사용)
    pos_msec = cap.get(cv2.CAP_PROP_POS_MSEC)

    # 20초 초과하면 중단
    if (target_frames is None and pos_msec >= 20000) or (target_frames is not None and idx >= target_frames):
        break

    # 저장
    out_path = os.path.join(OUTPUT_DIR, f"frame_{idx:06d}.jpg")
    ok = cv2.imwrite(out_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
    if ok:
        saved += 1
        if target_frames is not None:
            pbar.update(1)
        else:
            # 시간 기준일 때는 대략 진행률 표시
            pbar.update(1 if saved < pbar_total else 0)
    idx += 1

pbar.close()
cap.release()
print(f"완료: {saved}장 저장 → {OUTPUT_DIR}")


FPS: 29.970 | 총 프레임: 35841 | 저장 예정 프레임: 899


Extracting first 20s frames: 100% 899/899 [02:35<00:00,  5.76frame/s]


완료: 899장 저장 → ../data/test_patient_image/new_code/M04VISIT12_ori_frame


## sapiens 모델 초기화

In [31]:
```python
import mmpretrain  # VisionTransformer 등록

import mmcv
from mmdet.apis import init_detector, inference_detector
from mmpose.apis import init_model as init_pose_estimator, inference_topdown
from mmpose.utils import adapt_mmdet_pipeline
from mmpose.registry import VISUALIZERS
from mmpose.evaluation.functional import nms
from mmpose.structures import merge_data_samples, split_instances

# 경로
DATA_DIR = "../data/Patient_data/new_code/frames/M03_VISIT6_상지_frame"  # ← 경로만 변경
DET_CONFIG = "../sapiens/pose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person_no_nms.py"
DET_CKPT   = "../sapiens/pose/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth"
POSE_CONFIG= "../sapiens/pose/configs/sapiens_pose/coco/sapiens_0.3b-210e_coco-1024x768.py"
POSE_CKPT  = "../sapiens/pose/checkpoints/sapiens_0.3b/sapiens_0.3b_coco_best_coco_AP_796.pth"

# detector
detector = init_detector(DET_CONFIG, DET_CKPT, device="cuda:0")
detector.cfg = adapt_mmdet_pipeline(detector.cfg)

# pose estimator  (override_ckpt_meta 제거)
pose_estimator = init_pose_estimator(
    POSE_CONFIG, POSE_CKPT,
    device="cuda:0",
    cfg_options=dict(model=dict(test_cfg=dict(output_heatmaps=False)))
)

# 시각화기
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)
visualizer.set_dataset_meta(pose_estimator.dataset_meta, skeleton_style="mmpose")

print("모델 초기화 완료 ✅")
```


/opt/conda/lib/python3.10/importlib/__init__.py:169: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  _bootstrap._exec(spec, module)


Loads checkpoint by local backend from path: ../sapiens/pose/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Loads checkpoint by local backend from path: ../sapiens/pose/checkpoints/sapiens_0.3b/sapiens_0.3b_coco_best_coco_AP_796.pth
The model and loaded state dict do not match exactly

missing keys in source state_dict: head.deconv_layers.1.weight, head.deconv_layers.1.bias, head.deconv_layers.1.running_mean, head.deconv_layers.1.running_var, head.deconv_layers.4.weight, head.deconv_layers.4.bias, head.deconv_layers.4.running_mean, head.deconv_layers.4.running_var, head.conv_layers.1.weight, head.conv_layers.1.bias, head.conv_layers.1.running_mean, head.conv_layers.1.running_var, head.conv_layers.4.weight, head.conv_layers.4.bias, head.conv_layers.4.running_mean, head.conv_layers.4.running_var

모델 초기화 완료 ✅


/opt/conda/lib/python3.10/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmpose.visualization.local_visualizer.PoseLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(


## Sapiens 모델 실행
이때 Bbox는 5 Frame 마다 진행함.

In [32]:
# --- 30→15fps 다운샘플 + 주기적 검출 + 이전 포즈 bbox 재사용 (detector/pose_estimator/visualizer/DATA_DIR는 이전 셀에서 초기화됨) ---
import os, re, glob, json, numpy as np, mmcv, cv2                  # 표준/수치/영상/경로 라이브러리 임포트
from tqdm import tqdm                                               # 진행률 표시 바
from mmdet.apis import inference_detector                           # 사람 검출 API
from mmpose.apis import inference_topdown                           # 탑다운 포즈 추정 API
from mmpose.evaluation.functional import nms                        # NMS(겹침 박스 제거)
from mmpose.structures import merge_data_samples, split_instances   # 포즈 결과 병합/분리 유틸

OUTPUT_DIR = r"../data/test_patient_image/new_code/M04VISIT12_ori_json"

os.makedirs(OUTPUT_DIR, exist_ok=True)                               # 출력 폴더가 없으면 생성

def natural_key(s: str):                                            # 자연 정렬용 키 함수(파일명 속 숫자를 실제 숫자로 취급)
    base = os.path.basename(s)                                      # 파일명만 추출
    return [int(t) if t.isdigit() else t.lower()                    # 숫자는 int로, 문자는 소문자로 변환
            for t in re.split(r'(\d+)', base)]                      # 숫자/문자 구분하여 분리

all_frames = sorted(                                                # 원본 프레임 파일 경로 목록
    [p for p in glob.glob(os.path.join(DATA_DIR, "*"))              # DATA_DIR 내 모든 파일 중에서
     if p.lower().endswith((".jpg", ".png", ".jpeg"))],             # 이미지 확장자만 필터링
    key=natural_key)                                                # 자연 정렬 적용
assert all_frames, f"이미지 없음: {DATA_DIR}"                       # 이미지가 하나도 없으면 중단
frames = all_frames[::2]                                            # 30fps → 15fps: 2프레임마다 1장 선택
print(f"전체 {len(all_frames)}장 → 15fps용 {len(frames)}장 선택")     # 다운샘플링 결과 출력

def to_py(obj):                                                     # numpy/tensor-like → JSON 직렬화 가능 객체로 변환
    import numpy as _np                                             # 지역 임포트(이 함수 내부에서만 사용)
    if isinstance(obj, _np.ndarray): return obj.tolist()            # ndarray → list
    if isinstance(obj, (_np.floating,)): return float(obj)          # numpy float → float
    if isinstance(obj, (_np.integer,)):  return int(obj)            # numpy int → int
    if isinstance(obj, dict):  return {k: to_py(v) for k, v in obj.items()}  # dict 재귀 처리
    if isinstance(obj, (list, tuple)): return [to_py(v) for v in obj]        # list/tuple 재귀 처리
    return obj                                                      # 그 외는 그대로 반환

def clip_xyxy(xyxy, w, h):                                          # 박스 좌표를 이미지 경계 내로 클립
    x1, y1, x2, y2 = xyxy                                           # 좌상단(x1,y1), 우하단(x2,y2)
    return [max(0, x1), max(0, y1), min(w - 1, x2), min(h - 1, y2)] # 경계 밖이면 잘라서 반환

def bbox_from_keypoints(kpts, scores, thr=0.3, pad_scale=1.25, img_wh=None):  # 키포인트로부터 박스 추정
    valid = scores >= thr                                           # 신뢰도 임계 이상인 키포인트만 사용
    if valid.sum() == 0:                                            # 유효 키포인트가 없으면
        return None                                                 # 박스 생성 불가
    xy = kpts[valid]                                                # 유효 키포인트 좌표만 선택 (K',2)
    x1, y1 = xy.min(axis=0)                                         # 최소 x,y
    x2, y2 = xy.max(axis=0)                                         # 최대 x,y
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2                           # 중심 좌표
    w, h = max(2.0, x2 - x1), max(2.0, y2 - y1)                     # 최소 크기 보장(0 폭/높이 방지)
    w2, h2 = w * pad_scale / 2, h * pad_scale / 2                   # 패딩 반영한 반폭/반높이
    bx = [cx - w2, cy - h2, cx + w2, cy + h2]                       # 패딩된 박스 좌표
    if img_wh is not None:                                          # 이미지 크기가 주어졌다면
        bx = clip_xyxy(bx, img_wh[0], img_wh[1])                    # 경계 내로 클립
    return bx                                                       # 박스 반환

det_every = 5    # 몇 프레임마다 검출할지(15fps 기준 약 0.33초 간격)
det_thr   = 0.5  # 검출 score 임계값(단일 대상이면 높일수록 잡박스 감소)
nms_thr   = 0.5  # NMS IoU 임계값
kpt_thr   = 0.3  # 키포인트 표시/박스 생성 시 사용할 신뢰도 임계값               
pad_scale = 1.30 # 키포인트 박스 패딩 배율(살짝 여유 공간 확보)             
miss_max  = 2    # 포즈에서 bbox 재구성 실패 허용 횟수(연속)                     

prev_bbox = None # 이전 프레임에서 이어받은 박스(없으면 검출 강제)                     
miss_cnt  = 0    # 연속 bbox 생성 실패 횟수(임계 초과 시 재검출)                   
ok, fail  = 0, 0 # 처리 성공/실패 카운터                        

for i, img_path in enumerate(tqdm(frames)):                          # 다운샘플된 프레임을 순서대로 처리
    try:                                                             # 예외 처리 블록 시작
        img_bgr = cv2.imread(img_path)                               # 이미지 로드(BGR)
        if img_bgr is None:                                          # 로드 실패 시
            continue                                                 # 해당 프레임 스킵
        H, W = img_bgr.shape[:2]                                     # 이미지 높이/너비
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)           # RGB로 변환(mmdet/mmpose 사용)

        need_det = (i % det_every == 0) or (prev_bbox is None) or (miss_cnt > miss_max)  # 이번 프레임에서 검출 필요한지 판단
        if need_det:                                                 # 검출이 필요한 경우
            det = inference_detector(detector, img_rgb)              # 사람 검출 실행
            pred = det.pred_instances.cpu().numpy()                  # 결과를 numpy로 변환
            if len(pred.bboxes):                                     # 박스가 하나 이상이면
                bbs = np.concatenate((pred.bboxes,                   # (x1,y1,x2,y2) + score 결합
                                       pred.scores[:, None]), axis=1)
                keep = (pred.labels == 0) & (pred.scores > det_thr)  # 사람 클래스(0) + 점수 필터
                bbs = bbs[keep]                                      # 필터 적용
                if len(bbs) > 0:                                     # 남은 박스가 있으면
                    bbs = bbs[nms(bbs, nms_thr), :4]                 # NMS 적용 후 좌표만 취득
                if len(bbs) > 0:                                     # NMS 후에도 박스가 있으면
                    areas = (bbs[:, 2] - bbs[:, 0]) * (bbs[:, 3] - bbs[:, 1])  # 각 박스 면적 계산
                    prev_bbox = bbs[np.argmax(areas)].tolist()       # 가장 큰 박스를 선택(단일 대상 가정)
                    miss_cnt = 0                                     # 미스 카운트 리셋
                else:                                                # 박스가 사라졌다면
                    prev_bbox = None                                 # 추적 박스 초기화
            else:                                                    # 검출 결과가 비어 있으면
                prev_bbox = None                                     # 추적 박스 초기화

        bboxes_np = (np.array([prev_bbox], dtype=np.float32)         # 현재 프레임에서 사용할 bbox 배열
                     if prev_bbox is not None                        # 이전 박스가 있으면 1개 박스 사용
                     else np.empty((0, 4), dtype=np.float32))        # 없으면 빈 배열(포즈 추정 스킵)

        pose_results = inference_topdown(pose_estimator, img_rgb, bboxes_np)  # 포즈 추정 실행
        data_sample  = merge_data_samples(pose_results)              # 배치 결과 병합(단일/다중 인스턴스 대응)

        inst = data_sample.get('pred_instances', None)               # 예측 인스턴스 딕셔너리 취득
        if inst is not None and len(inst.get('keypoints', [])) > 0:  # 포즈가 하나 이상 감지되면
            kpts_all    = inst['keypoints']                          # (N,K,2) 모든 사람의 키포인트 좌표
            kscores_all = inst['keypoint_scores']                    # (N,K)  모든 사람의 키포인트 신뢰도
            idx = int(np.argmax(kscores_all.mean(axis=1)))           # 평균 신뢰도가 가장 높은 사람 선택
            kpts, kscores = kpts_all[idx], kscores_all[idx]          # 선택된 사람의 키포인트/점수
            nb = bbox_from_keypoints(kpts, kscores,                  # 키포인트로 다음 프레임용 bbox 생성
                                     thr=kpt_thr, pad_scale=pad_scale,
                                     img_wh=(W, H))
            if nb is not None:                                       # 박스 생성에 성공하면
                prev_bbox = nb                                       # 추적 박스 업데이트
                miss_cnt = 0                                         # 미스 카운트 리셋
            else:                                                    # 박스 생성 실패 시
                miss_cnt += 1                                        # 미스 카운트 증가
        else:                                                        # 포즈가 전혀 없으면
            miss_cnt += 1                                            # 미스 카운트 증가


        # ===== [이미지 시각화/저장 비활성화] =====
        # visualizer.add_datasample(                                   # 시각화기에게 결과 렌더링 요청
        #     name="result",                                           # 샘플 이름(임의)
        #     image=img_rgb,                                           # 원본 RGB 이미지
        #     data_sample=data_sample,                                 # 포즈 예측 결과
        #     draw_gt=False,                                           # GT(정답) 미표시
        #     draw_bbox=False,                                          # bbox 그리기
        #     kpt_thr=kpt_thr,                                         # 키포인트 표시 임계값
        #     show=False)                                              # 노트북 창 표시 안 함(파일로만 저장)

        # out_img = os.path.join(OUTPUT_DIR, os.path.basename(img_path))  # 출력 이미지 경로
        # mmcv.imwrite(mmcv.rgb2bgr(visualizer.get_image()), out_img)     # 렌더 결과(BGR) 저장

        # ===== JSON만 저장 =====
        if inst is not None:
            inst_list = split_instances(inst)
            payload = dict(meta_info=pose_estimator.dataset_meta,
                           instance_info=inst_list)
        
            # ✅ out_img 쓰지 말고 직접 파일명으로 경로 생성
            base = os.path.splitext(os.path.basename(img_path))[0]
            json_path = os.path.join(OUTPUT_DIR, base + ".json")
        
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(to_py(payload), f, ensure_ascii=False, indent=2)


        ok += 1                                                       # 성공 카운트 증가

    except Exception as e:                                            # 예외 발생 시
        fail += 1                                                     # 실패 카운트 증가
        print("에러:", os.path.basename(img_path), "->", e)           # 파일명과 에러 메시지 출력

print(f"완료. 성공 {ok}, 실패 {fail}, 출력: {OUTPUT_DIR}")            # 전체 처리 요약 출력


전체 899장 → 15fps용 450장 선택


100% 450/450 [04:26<00:00,  1.69it/s]

완료. 성공 450, 실패 0, 출력: ../data/test_patient_image/new_code/M04VISIT12_ori_json


## Overlay image 생성
json과 ori_img를 통해 overlay iamge 생성

In [33]:
# --- 15fps 예측(JSON) → 30fps 업샘플: JSON 저장 없이 '오버레이 이미지'만 저장 ---
import os, re, glob, json            # 경로/정렬/파일검색/JSON
import cv2                           # OpenCV (이미지 로드/저장, 그리기)
import numpy as np                   # 수치 계산
from tqdm import tqdm                # 진행률 표시

# ===== 경로 =====
DATA_DIR     = "../data/test_patient_image/new_code/M04VISIT12_ori_frame"          # 원본 30fps 프레임 폴더
SRC_JSON_DIR = "../data/test_patient_image/new_code/M04VISIT12_ori_json"     # 15fps 결과 JSON 폴더
OUT_VIS_DIR  = "../data/test_patient_image/new_code/M04VISIT12_ori_vis"  # 30fps 오버레이 저장
os.makedirs(OUT_VIS_DIR, exist_ok=True)

# ===== 자연 정렬 =====
def natural_key(s: str):
    base = os.path.basename(s)
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', base)]

# ===== 30fps 전체 프레임 목록 =====
all_frames = sorted(
    [p for p in glob.glob(os.path.join(DATA_DIR, "*")) if p.lower().endswith((".jpg",".png",".jpeg"))],
    key=natural_key
)
assert all_frames, f"원본 프레임이 없습니다: {DATA_DIR}"

# ===== 15fps JSON 맵 =====
src_json_files = sorted(glob.glob(os.path.join(SRC_JSON_DIR, "*.json")), key=natural_key)
assert src_json_files, f"15fps JSON이 없습니다: {SRC_JSON_DIR}"
src_json_map = {os.path.splitext(os.path.basename(p))[0]: p for p in src_json_files}

# ===== 시각화 파라미터 (단일 색) =====
KPT_THR         = 0.05                # 키포인트 신뢰도 임계값 (필요시 0.2~0.3으로 올려 노이즈 제거)
KEYPOINT_COLOR  = (0, 255, 0)         # 점 색 (BGR)
SKELETON_COLOR  = (255, 128, 0)       # 선 색 (BGR)
RADIUS          = 4                   # 점 반지름
THICKNESS       = 2                   # 선 두께
ANTI_ALIAS      = cv2.LINE_AA         # 안티앨리어싱

# ===== 스켈레톤 링크 얻기 =====
def get_links(meta: dict):
    if "skeleton_links" in meta:
        return meta["skeleton_links"]
    if "skeleton" in meta:
        return meta["skeleton"]
    return []

# ===== 한 인스턴스 그리기 (단일 색, bbox X) =====
def draw_instance_uniform(img_bgr, instance: dict, links, kpt_thr=0.05):
    kpts = np.array(instance["keypoints"], dtype=np.float32)
    ksc  = np.array(instance["keypoint_scores"], dtype=np.float32)
    # 키포인트(점)
    for xy, sc in zip(kpts, ksc):
        if sc < kpt_thr:
            continue
        x, y = int(xy[0]), int(xy[1])
        cv2.circle(img_bgr, (x, y), RADIUS, KEYPOINT_COLOR, -1, lineType=ANTI_ALIAS)
    # 스켈레톤(선)
    for link in links:
        if isinstance(link, dict) and "link" in link:
            i, j = link["link"]
        elif isinstance(link, (list, tuple)) and len(link) == 2:
            i, j = link
        else:
            continue
        if i >= len(kpts) or j >= len(kpts):
            continue
        if ksc[i] < kpt_thr or ksc[j] < kpt_thr:
            continue
        p1 = (int(kpts[i][0]), int(kpts[i][1]))
        p2 = (int(kpts[j][0]), int(kpts[j][1]))
        cv2.line(img_bgr, p1, p2, SKELETON_COLOR, THICKNESS, lineType=ANTI_ALIAS)

# ===== 메인: 30fps 모든 프레임에 대해 오버레이 이미지 저장 (JSON은 저장하지 않음) =====
last_json_data = None
written_img, skipped = 0, 0

for fpath in tqdm(all_frames, desc="Overlay 30fps", unit="frame"):
    base = os.path.splitext(os.path.basename(fpath))[0]           # 예: frame_000123
    # 15fps JSON 있으면 로드, 없으면 직전 JSON을 사용(Zero-Order Hold)
    src_json_path = src_json_map.get(base, None)
    if src_json_path is not None:
        with open(src_json_path, "r", encoding="utf-8") as f:
            json_data = json.load(f)
        last_json_data = json_data
    else:
        if last_json_data is None:
            # (초반부 예외 처리) 앞으로 보이는 첫 JSON 1회용 사용
            next_json_path = None
            for nb in sorted(src_json_map.keys(), key=natural_key):
                if nb > base:
                    next_json_path = src_json_map[nb]
                    break
            if next_json_path is None:
                skipped += 1
                continue
            with open(next_json_path, "r", encoding="utf-8") as f:
                json_data = json.load(f)
        else:
            json_data = last_json_data

    # 원본 프레임 로드
    img_bgr = cv2.imread(fpath)
    if img_bgr is None:
        skipped += 1
        continue

    # 스켈레톤/키포인트 그리기
    meta      = json_data.get("meta_info", {})
    instances = json_data.get("instance_info", [])
    links     = get_links(meta)
    for inst in instances:
        draw_instance_uniform(img_bgr, inst, links, kpt_thr=KPT_THR)

    # 오버레이 이미지 저장
    out_img_path = os.path.join(OUT_VIS_DIR, base + ".jpg")
    if cv2.imwrite(out_img_path, img_bgr):
        written_img += 1
    else:
        skipped += 1

print(f"완료: 오버레이 이미지 {written_img}개 저장, 스킵 {skipped}개")


Overlay 30fps: 100% 899/899 [03:49<00:00,  3.93frame/s]

완료: 오버레이 이미지 899개 저장, 스킵 0개


## MP4 생성

In [34]:
# --- 업샘플된 30fps 시각화 프레임 → 30fps MP4 (tqdm 진행률, 지정 경로 저장) ---
import os, re, glob, cv2
from tqdm import tqdm

# 업샘플된 30fps 프레임들이 있는 폴더 (그림 이미지들)

VIS_DIR  = "../data/test_patient_image/new_code/M04VISIT12_ori_vis"  # 30fps 오버레이 저장

# MP4 저장 폴더와 파일명
SAVE_ROOT = "../data/test_patient_image/new_code"
os.makedirs(SAVE_ROOT, exist_ok=True)
SAVE_MP4  = os.path.join(SAVE_ROOT, "M04VISIT12_ori_output.mp4")

def natural_key(s: str):
    base = os.path.basename(s)
    return [int(t) if t.isdigit() else t.lower() for t in re.split(r'(\d+)', base)]

# 프레임 수집 (자연 정렬)
frames = sorted(
    [p for p in glob.glob(os.path.join(VIS_DIR, "*")) if p.lower().endswith((".jpg", ".png", ".jpeg"))],
    key=natural_key
)
assert frames, f"프레임 이미지가 없습니다: {VIS_DIR}"

# 비디오 라이터 초기화
first = cv2.imread(frames[0])
assert first is not None, f"첫 프레임 로드 실패: {frames[0]}"
h, w = first.shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # 호환성 좋은 코덱
writer = cv2.VideoWriter(SAVE_MP4, fourcc, 30, (w, h))

# 인코딩 루프 (tqdm 진행률)
for f in tqdm(frames, desc="Encoding 30fps MP4", unit="frame"):
    img = cv2.imread(f)
    if img is None:
        print(f"[경고] 프레임 로드 실패: {f}")
        continue
    if img.shape[:2] != (h, w):
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_AREA)
    writer.write(img)

writer.release()
print("30fps 비디오 저장 완료:", SAVE_MP4)


Encoding 30fps MP4: 100% 899/899 [02:18<00:00,  6.50frame/s]


30fps 비디오 저장 완료: ../data/test_patient_image/new_code/M04VISIT12_ori_output.mp4
